In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
print(K.image_data_format())

channels_last


In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    num_class = 2
    
    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, num_feature), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(num_feature):
                X_train[sam_count,i] = float(data_train[index,i])
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, num_feature), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(num_feature):
            X_test[index,i] = float(data_test[index,i])
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    #X_train_new = np.empty((nrow_train_dup, 224, 224, 3))
    #Y_train_new = Y_train
    
    #for index in range(nrow_train_dup):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
    #    for c in range(3):
    #        X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    #X_test_new = np.empty((nrow_test, 224, 224, 3))
    #Y_test_new = Y_test
    
    #for index in range(nrow_test):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
    #    for c in range(3):
    #        X_test_new[index,:,:,c] = tmp_new[:,:,0]
            
    #del X_train, X_test
    #gc.collect()

    return X_train,Y_train,X_test,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,length,classes):
    input_shape = (channel,length)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (length,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    #model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    #model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    #model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(2400,activation=LeakyRelu,input_dim=length,name="fc1"))
    model.add(Dense(600,activation=LeakyRelu,name="fc2"))
    model.add(Dense(768,activation=LeakyRelu,name="fc3"))
    model.add(Dense(192,activation=LeakyRelu,name="fc4"))
    model.add(Dense(classes,activation="softmax",name="fc5"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_label = model.predict_classes(sam_data, batch_size=32, verbose=1)
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)
                
                num_feature = X_train.shape[1]

                # # model
                #model = ResNet50(
                #    weights=None,
                #    classes=2
                #)

                ##
                model = neural(1,num_feature,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                model.fit(X_train, Y_train, epochs=50, batch_size=16)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2760 - acc: 0.9333
Epoch 2/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2457 - acc: 0.9333
Epoch 3/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2533 - acc: 0.9333
Epoch 4/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2368 - acc: 0.9340
Epoch 5/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2341 - acc: 0.9333
Epoch 6/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2417 - acc: 0.9333
Epoch 7/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2301 - acc: 0.9326
Epoch 8/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2215 - acc: 0.9333
Epoch 9/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2121 - acc: 0.9326
Epoch 10/50

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 2
Epoch 1/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2948 - acc: 0.9326
Epoch 2/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2500 - acc: 0.9333
Epoch 3/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2647 - acc: 0.9333
Epoch 4/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2456 - acc: 0.9333A: 1s - l
Epoch 5/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2445 - acc: 0.9326
Epoch 6/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2268 - acc: 0.9333
Epoch 7/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2733 - acc: 0.9333
Epoch 8/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 9/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333A: 

1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333A: 1s - los
Epoch 31/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 32/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 33/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 34/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 35/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 36/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 37/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 38/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 39/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0745 - acc: 0.9333
Epoch 40/50
1410/1410

1410/1410 [==============================] - 2s 1ms/step - loss: 0.2115 - acc: 0.9340
Epoch 8/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2135 - acc: 0.9340
Epoch 9/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2013 - acc: 0.9348
Epoch 10/50
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2500 - acc: 0.9376
Epoch 11/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 12/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 13/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 14/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 15/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 16/50
1410/1410 [==============================] - 2s 1ms/step - loss: 1.0631 - acc: 0.9340
Epoch 17/50
1410/1410 [===========

...

5238/5238 [==============================] - 8s 2ms/step - loss: 0.3284 - acc: 0.8597
Epoch 12/50
5238/5238 [==============================] - 10s 2ms/step - loss: 0.2381 - acc: 0.9143
Epoch 13/50
5238/5238 [==============================] - 10s 2ms/step - loss: 0.1988 - acc: 0.9273
Epoch 14/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.1378 - acc: 0.9549
Epoch 15/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.1158 - acc: 0.9607
Epoch 16/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.1165 - acc: 0.9603
Epoch 17/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.1020 - acc: 0.9662
Epoch 18/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.1120 - acc: 0.9622
Epoch 19/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.0819 - acc: 0.9756
Epoch 20/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.0613 - acc: 0.9813
Epoch 21/50
5238/5238 [=======

5238/5238 [==============================] - 8s 2ms/step - loss: 2.6194e-06 - acc: 1.0000
Epoch 41/50
5238/5238 [==============================] - 8s 2ms/step - loss: 1.5921e-06 - acc: 1.0000
Epoch 42/50
5238/5238 [==============================] - 8s 2ms/step - loss: 1.0781e-06 - acc: 1.0000
Epoch 43/50
5238/5238 [==============================] - 8s 2ms/step - loss: 8.0007e-07 - acc: 1.0000
Epoch 44/50
5238/5238 [==============================] - 8s 2ms/step - loss: 6.1563e-07 - acc: 1.0000
Epoch 45/50
5238/5238 [==============================] - 8s 2ms/step - loss: 4.7462e-07 - acc: 1.0000
Epoch 46/50
5238/5238 [==============================] - 8s 2ms/step - loss: 4.2201e-07 - acc: 1.0000
Epoch 47/50
5238/5238 [==============================] - 9s 2ms/step - loss: 3.6602e-07 - acc: 1.0000
Epoch 48/50
5238/5238 [==============================] - 9s 2ms/step - loss: 3.0641e-07 - acc: 1.0000
Epoch 49/50
5238/5238 [==============================] - 8s 2ms/step - loss: 2.6801e-07 - acc:

5238/5238 [==============================] - 8s 2ms/step - loss: 0.0527 - acc: 0.9836
Epoch 17/50
5238/5238 [==============================] - 8s 2ms/step - loss: 0.0464 - acc: 0.9836
Epoch 18/50
5238/5238 [==============================] - 8s 2ms/step - loss: 0.0458 - acc: 0.9828
Epoch 19/50
5238/5238 [==============================] - 8s 2ms/step - loss: 0.0430 - acc: 0.9866
Epoch 20/50
5238/5238 [==============================] - 8s 2ms/step - loss: 0.0271 - acc: 0.9906
Epoch 21/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.0163 - acc: 0.9956
Epoch 22/50
5238/5238 [==============================] - 9s 2ms/step - loss: 0.0508 - acc: 0.9842
Epoch 23/50
5238/5238 [==============================] - 8s 2ms/step - loss: 0.0234 - acc: 0.9906
Epoch 24/50
5238/5238 [==============================] - 8s 2ms/step - loss: 1.5199e-04 - acc: 1.0000
Epoch 25/50
5238/5238 [==============================] - 8s 2ms/step - loss: 1.2155e-05 - acc: 1.0000
Epoch 26/50
5238/5238 [=

5266/5266 [==============================] - 9s 2ms/step - loss: 6.0661e-07 - acc: 1.0000
Epoch 44/50
5266/5266 [==============================] - 9s 2ms/step - loss: 4.9208e-07 - acc: 1.0000
Epoch 45/50
5266/5266 [==============================] - 8s 2ms/step - loss: 4.2224e-07 - acc: 1.0000
Epoch 46/50
5266/5266 [==============================] - 8s 2ms/step - loss: 3.6008e-07 - acc: 1.0000
Epoch 47/50
5266/5266 [==============================] - 8s 2ms/step - loss: 3.1367e-07 - acc: 1.0000
Epoch 48/50
5266/5266 [==============================] - 8s 2ms/step - loss: 2.7058e-07 - acc: 1.0000
Epoch 49/50
5266/5266 [==============================] - 8s 2ms/step - loss: 2.3660e-07 - acc: 1.0000
Epoch 50/50
1411/1411 [==============================] - 1s 927us/step
[1317.   94.]
156/156 [==============================] - 0s 275us/step
[146.  10.]
X_train shape :  (5266, 474)
Y_train shape :  (5266, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
1 1 1 9
Epoch 1/50
5266/5266 [======

Epoch 21/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0307 - acc: 0.9907
Epoch 22/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0660 - acc: 0.9795
Epoch 23/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0326 - acc: 0.9877
Epoch 24/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0525 - acc: 0.9789
Epoch 25/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0331 - acc: 0.9884
Epoch 26/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0189 - acc: 0.9941
Epoch 27/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0633 - acc: 0.9827
Epoch 28/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0151 - acc: 0.9956
Epoch 29/50
5266/5266 [==============================] - 8s 2ms/step - loss: 0.0156 - acc: 0.9951
Epoch 30/50
5266/5266 [==============================] - 8s 1ms/step - loss: 0.2112 - acc: 0.9392
Epoch 31/50
5266/526

In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[ 0.01514813  0.01492755  0.01486372 ...  0.00857476 -0.00436995
  -0.00757773]
 [-0.00032011  0.00909576  0.0037819  ...  0.01969244  0.00399345
  -0.01421445]
 [-0.00353804  0.00738441  0.00642808 ...  0.00350748  0.01241723
   0.01060618]
 ...
 [ 0.00410879  0.01063525 -0.01757033 ...  0.00444562 -0.00209475
  -0.00322124]
 [-0.00043965  0.01629578  0.00783747 ...  0.00279593 -0.00999914
  -0.01270919]
 [ 0.01259797  0.0029113   0.00486612 ...  0.00593103 -0.01530847
   0.00264231]]
(1411, 474)
(1411, 474)
(1411, 474)
(2822, 474)


In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1.317e+03 1.000e+00]
[93.  0.]
[1.000e+00 1.317e+03]
[ 0. 93.]
[1313.   43.]
[51.  4.]
[  43. 1313.]
[ 4. 51.]
[1317.   10.]
[84.  0.]
[  10. 1317.]
[ 0. 84.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[143.   3.]
[7. 3.]
[  3. 143.]
[3. 7.]
[145.   1.]
[9. 1.]
[  1. 145.]
[1. 9.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]


In [12]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)